In [ ]:
from nocodb_base import fetch_all_rows_from_nocodb, update_row
from file_utils import write_json, write_tsv, read_tsv
import re
import time


In [ ]:
# Configuration
xc_token = ""
table_id = "mmhhg1qgufpq1su"
view_id = "vwy56y8bhpz7qmss"
noco_base_url = ""
headers = {
    'accept': 'application/json',
    'xc-token': xc_token,
}
seed_file = "add_url_lists/cloudfront/combined.txt"

In [ ]:
rows_data = fetch_all_rows_from_nocodb(noco_base_url, headers, table_id, view_id)
#print(rows_data)

In [ ]:
from file_utils import write_json, write_tsv
output_file = "rows.txt"
write_tsv(output_file, rows_data)

In [ ]:
rows_data = read_tsv("rows.txt")

In [ ]:
from urllib.parse import urlparse, urlunparse, quote

def normalize_url(url):
    url = url.strip()
    parsed = urlparse(url)

    scheme = parsed.scheme.lower()
    netloc = parsed.netloc.lower()

    # Lowercase path, strip trailing slash
    path = parsed.path.rstrip('/').lower()

    # Keep query string if present (as-is)
    query = parsed.query

    # Rebuild the normalized URL
    normalized = urlunparse((scheme, netloc, path, '', query, ''))

    # Encode safely (optional but helpful if comparing to encoded index)
    return quote(normalized, safe=':/?&=%')

In [ ]:
urls_list = {}
with open(seed_file, 'rt', encoding='utf-8') as f:
    for line in f:
        url = normalize_url(line)
        urls_list[url] = True

In [ ]:
def update_row_defined_data(noco_base_url, headers, table_id, record_id, index_status,index_check_field):
    """Update the index_check column in NocoDB."""
    request_url = (
        f"{noco_base_url}/api/v2/tables/{table_id}/records"
    )
    data = {
        "Id": record_id,
        index_check_field: index_status,
    }
    #print(request_url)
    #print(data)
    try:
        response = requests.patch(request_url, headers=headers, json=data)
        #print(response)
        #print("Response Status Code:", response.status_code)
        #print("Response Text:", response.text)
        
        if response.status_code == 200:
            pass
            #print(f"Updated record {record_id} -> {index_check_field}: {index_status}")
        else:
            print(f"Failed to update record {record_id}. Status: {response.status_code}")
            print(f"Response text: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error updating record {record_id}: {e}")

In [ ]:
import requests

# Process rows and add index status
urls = []
select_field = "new"
new_select_item = "added"

# Build mapping from normalized URL -> row
url_to_row = {}
for row in rows_data:
    url = row.get("URL")
    if not url:
        continue
    normalized = normalize_url(url)
    url_to_row[normalized] = row


for url in urls_list:
    #print(url)
    normalized_url_seedlist = normalize_url(url)
    #print(normalized_url_seedlist)

    if normalized_url_seedlist in url_to_row:
        #print("is in table")
        
        ### comment following lines, if item shouldn't be updated, when matched
        row = url_to_row[normalized_url_seedlist]
        record_id = row.get("Id")
        select_data = row.get(select_field) or ""
        current_items = [item.strip() for item in select_data.split(',') if item.strip()]

        if new_select_item not in current_items:
            current_items.append(new_select_item)
            new_select = ','.join(current_items)
            #print(new_select)
            update_row_defined_data(noco_base_url, headers, table_id, record_id, new_select, select_field)  
        else:
            new_select = current_items
            update_row_defined_data(noco_base_url, headers, table_id, record_id, new_select, select_field)  

    else:
        #print("not in table")
        new_select = new_select_item
        request_url = (
            f"{noco_base_url}/api/v2/tables/{table_id}/records"
        )
        data = {
            "URL": normalized_url_seedlist,
            select_field: new_select,
        }
        
        try:
            response = requests.post(request_url, headers=headers, json=data)
            if response.status_code == 200:
                pass
                #print(f"created record -> new value: {normalized_url_seedlist}")
            else:
                print(f"Failed to create. Status: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error creating record: {e}")
    

# Save updated rows
#output_file = "cloudfront_not_in_index.json"
output_file = "add_seed.json"

write_json(output_file, urls)
print(f"Updated rows saved to {output_file}")